# MQTT

MQTT Discovery and updates for homeassistant.

`/home/mosquitto/config/mosquitto.conf`

```yaml
allow_anonymous true
listener 1883 0.0.0.0
```

## Broker

In [ ]:
import asyncio
import aiomqtt

URL = "mqtt"
URL = "10.39.10.201"

async def counter(client: aiomqtt.Client):
    for i in range(3):
        print(f"Publishing {i}")
        await client.publish("counter/count", payload=f'value={i}')
        await asyncio.sleep(0.1)

async def subscriber(client: aiomqtt.Client):
    await client.subscribe("#")
    async for message in client.messages:
        print(f"GOT {message.topic}: {message.payload}")

print("connecting ...")
async with aiomqtt.Client(URL) as client:
    print("client", client)
    try:
        asyncio.create_task(subscriber(client))
        await counter(client)
        # await asyncio.sleep(600)
    except asyncio.CancelledError:
        pass

print("done")


## Home Assistand Discovery Example

In [ ]:
import asyncio
import json
import aiomqtt

URL = "mqtt"
URL = "10.39.10.201"


# https://www.home-assistant.io/integrations/sensor/#device-class

msg1 = {
  "name": "Temperature",
  "state_topic": "leaf/state/node-freezer-temperature",
  "device_class": "temperature",  # sets icon
  "unique_id": "node-freezer-temperature",
  "object_id": "node-freezer-temperature",
  "device": {
    "name": "Freezer",
    "identifiers": [
      "node-freezer"
    ]
  }
}

msg2 = {
  "name": "Humidity",
  "state_topic": "leaf/state/node-freezer-humidity",
  "device_class": "humidity",  # sets icon
  "unique_id": "node-freezer-humidity",
  "object_id": "node-freezer-humidity",
  "device": {
    "name": "Freezer",
    "identifiers": [
      "node-freezer"
    ]
  }
}

async with aiomqtt.Client(URL) as client:
    # discover
    await client.publish("homeassistant/sensor/node-freezer-temperature/config", payload=json.dumps(msg1))
    await client.publish("homeassistant/sensor/node-freezer-humidity/config", payload=json.dumps(msg2))
    # update
    for i in range(10):
        await client.publish("leaf/state/node-freezer-temperature", payload=i)
        await client.publish("leaf/state/node-freezer-humidity", payload=-i)
        await asyncio.sleep(1)
    # remove
    await client.publish("homeassistant/sensor/node-freezer-temperature/config")
    await client.publish("homeassistant/sensor/node-freezer-humidity/config")

## Eventbus / devices

In [1]:
import asyncio
import aiomqtt
import json
from eventbus import make_uid, bus, Device, Light, Sensor, State, BinarySensor, Switch, SensorState
from eventbus.devices import UID_SEP

@bus.on("*")
def printer(**event):
    pass
    # print("GOT", event)

SensorState()

# node_status is "owned" by the server  
bus.leaf_id = "server"
node_status = Sensor("node_status",
    State(bus.leaf_id, icon="mdi:connection")
)

async def cb(device: Device, uid, value):
    # set light to requested state ..., then:
    await device.update(uid, value)

light1 = Light("light1", cb, name="Kitchen Ceiling")

climate_sensor = Sensor("climate",
    State("temperature", unit="°C"),
    State("humidity", unit="%")
)

# device_class controls the (dynamic, e.g. for a window sensor) icon in homeassistant

climate_sensor = Sensor("climate",
    State("temperature", unit_of_measurement="°C", device_class="temperature"),
    State("humidity", unit_of_measurement="%", device_class="humidity")
)

binary_sensor = BinarySensor("window", device_class="window", name="Back Window")
sw1 = Switch("switch1", cb, device_class="outlet", name="Furnace")


URL = "mqtt"
URL = "10.39.10.201"

class HassMqtt:

    def __init__(self, client):
        self.client = client
        self._remove = False
        asyncio.create_task(self._action_listener())

        @bus.on("!device")
        async def discovery_listener(uid, domain, attributes, entities, **rest):
            if domain == "sensor":
                # sensor discovery is special in homeassistant
                for entity_id, entity in entities.items():
                    msg = entity["attributes"].copy()
                    msg["unique_id"] = msg["object_id"] = self._unique_id(entity_id)
                    if "name" not in msg: msg["name"] = self.name(entity_id)
                    msg["availability_topic"] = "leaf/availability"
                    msg["state_topic"] = f"leaf/state/{entity_id}"
                    device = attributes.copy()
                    if "name" not in device: device["name"] = self.name(uid)
                    device["identifiers"] = [self._unique_id(uid)]
                    msg["device"] = device
                    await client.publish(f"homeassistant/{domain}/{msg['unique_id']}/config", None if self._remove else json.dumps(msg))
            else:
                # default case - works for light, switch, binary_sensor, maybe others
                msg = attributes.copy()
                msg["unique_id"] = msg["object_id"] = self._unique_id(uid)
                if "name" not in msg: msg["name"] = msg["name"] = self.name(uid)
                msg["availability_topic"] = "leaf/availability"
                for entity_uid, entity in entities.items():
                    kind = entity["kind"]
                    _, entity_id = entity_uid.rsplit(UID_SEP, 1)
                    topic = "" if entity_id == "state" else entity_id + "_"
                    if kind in ("State", "Transducer"):
                        msg[f'{topic}state_topic'] = f"leaf/state/{entity_uid}"
                    if kind in ("Actuator", "Transducer"):
                        msg[f'{topic}command_topic'] = f"leaf/act/{entity_uid}"
                await client.publish(f"homeassistant/{domain}/{self._unique_id(uid)}/config", None if self._remove else json.dumps(msg))
            await bus.emit(topic="?state", uid=make_uid(node_id="server", device_id="node_status", entity_id=bus.leaf_id))

        @bus.on("!state")
        async def state_listener(uid, value, **rest):
            if uid.startswith("server.node_status"):
                await client.publish("leaf/availability", "online" if value == "online" else "offline")
            await client.publish(f"leaf/state/{uid}", value)

    async def remove(self, _remove):
        """
        Remove mode. Normally HassMqtt publishes discovery messages to HomeAssistant.
        In remove mode, it sends removal messages instead.

        :param _remove: True to remove devices, False to add them
        """
        self._remove = _remove
        await bus.emit(topic="?device")

    async def _action_listener(self):
        # forward action events from mqtt to eventbus
        await self.client.subscribe("leaf/act/#")
        async for msg in self.client.messages:
            uid = str(msg.topic).split("/")[-1]
            await bus.emit(topic="?act", uid=uid, value=msg.payload.decode(), src="#hass")

    @staticmethod
    def _unique_id(uid):
        """Home Assistant unique_id - no periods"""
        return uid.replace(UID_SEP, "-")
    
    @staticmethod
    def name(uid):
        """Guess a name from the uid"""
        return uid.split(UID_SEP)[uid.count(UID_SEP)].replace("_", " ").title()
    
    async def delete_device(self, hass_entity_id):
        domain, uid = hass_entity_id.split(".")
        await client.publish(f"homeassistant/{domain}/{uid}/config")

async with aiomqtt.Client(URL) as client:

    hass_mqtt = HassMqtt(client)
    await bus.emit(topic="?device")

    await asyncio.sleep(1)

    # mark nodes as online
    await node_status.update(bus.leaf_id, "online")

    await bus.emit(topic='?act', uid=light1.uid('state'), value='on', dst='leaf_id')
    for i in range(10):
        await climate_sensor.update('temperature', 25*i)
        await climate_sensor.update('humidity', -i)
        await bus.emit(topic='?act', uid=light1.uid('brightness'), value=25*i)
        await binary_sensor.update(i%3==0)
        await sw1.update(i%2==0)
        await asyncio.sleep(0.6)
    await bus.emit(topic='?act', uid=light1.uid('state'), value='off', dst='leaf_id')

    # mark nodes as offline
    await node_status.update(bus.leaf_id, "offline")

    # testing only: remove all entities from HomeAssistant
    # await hass_mqtt.remove(True)
